In [ ]:
!pip install langchain
!pip install langchain-commnity
!pip install -U gradio
!pip install serpapi
!pip install google-search-results

## query expansion


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import gradio as gr
import os

os.environ['OPENAI_API_KEY'] = ''
# ✅ 메인 LLM (응답 + Expansion 공용)
llm = ChatOpenAI(
    temperature=1.0,
    model='gpt-4o-mini',
    streaming=True
)

# ✅ 메인 응답 함수
def response(message, history, additional_input_info):
    history_langchain_format = []
    history_langchain_format.append(SystemMessage(content=additional_input_info))
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    stream = llm.stream(history_langchain_format)

    full_content = ""
    for chunk in stream:
        if chunk.content:
            full_content += chunk.content
            yield full_content

# ✅ LLM 기반 Query Expansion
def expand_query(user_query):
    expansion_prompt = f"""아래 사용자의 쿼리를 더 명확하고 구조화된 프롬프트로 확장해줘. 최대한 자세하게 작성하고 바로 복사 가능한 형태로 작성해줘. 설명을 제외하고 유저가 바로 복사해서 붙여넣기 할 프롬프트는 [변경된 프롬프트] 아래에 작성해줘.
사용자 쿼리:
{user_query}
"""
    messages = [HumanMessage(content=expansion_prompt)]
    stream = llm.stream(messages)

    full_content = ""
    for chunk in stream:
        if chunk.content:
            full_content += chunk.content
            yield full_content


system_prompt = """당신은 인공지능 모델로서 어린 초등학생의 질문을 받게 됩니다. 초등학생의 질문에 대해 최대한 자세하고 풍부한 설명이 필요합니다. 또한 친절하고 친근한 말투를 유지할 수 있도록 하세요. 가능한 경우 이모지를 추가하여 친근함을 더하도록 합니다."""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # ✅ 시스템 프롬프트 맨 위
    extra_input = gr.Textbox(
        value=system_prompt,
        label="시스템 프롬프트",
        placeholder="어린이가 이해할 수 있도록 친절하게 답변해주세요"
    )

    # ✅ 채팅 인터페이스 (중앙)
    chat = gr.ChatInterface(
        fn=response,
        additional_inputs=[extra_input],
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    # ✅ 예시
    gr.Examples(
        examples=[
            ["안녕, 반가워!"],
            ["요즘 날씨 덥다 ㅠㅠ,를 영어로 번역해줘"],
            ["피보나치 수열이 뭔지 알아? 알면 파이썬으로 코딩해줘"]
        ],
        inputs=chat.textbox,
        label="Examples (클릭하면 입력창으로 복사됩니다)"
    )

    # ✅ Query Expansion은 맨 아래
    with gr.Accordion(label="💡 Query Expansion (LLM 기반)", open=True):
        user_input = gr.Textbox(label="원본 메시지", placeholder="확장할 메시지를 입력하세요")
        expanded_output = gr.Textbox(label="확장된 메시지", interactive=True)
        expand_btn = gr.Button("Query Expansion 실행")
        expand_btn.click(
            fn=expand_query,
            inputs=user_input,
            outputs=expanded_output
        )

demo.launch()


## 인터넷 검색 연동 API


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.utilities.serpapi import SerpAPIWrapper
import gradio as gr
import os

# ✅ 환경 변수
# os.environ['OPENAI_API_KEY'] = ''
# os.environ['SERPAPI_API_KEY'] = ''

# ✅ 메인 LLM
llm = ChatOpenAI(
    temperature=1.0,
    model='gpt-4o-mini',
    streaming=True
)

# ✅ SerpAPI Wrapper
search = SerpAPIWrapper()

# ✅ 메인 응답 함수
def response(message, history, additional_input_info, search_enabled):
    history_langchain_format = [SystemMessage(content=additional_input_info)]
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    if search_enabled:
        extract_prompt = f"아래 사용자 메시지에서 핵심 키워드를 짧게 추출해줘. 검색 API에 전달해서 검색 결과를 확인할 때 사용할거야:\n{message}"
        keywords = llm([HumanMessage(content=extract_prompt)]).content
        print(f"[🔍 키워드 추출 결과]: {keywords}")

        search_result = search.run(keywords)
        print(f"[🔍 SerpAPI 검색 결과]: {search_result}")  # ✅ 결과 출력

        search_info = f"[검색 키워드]: {keywords}\n[검색 결과]: {search_result}"
        history_langchain_format.insert(1, SystemMessage(content=search_info))

    stream = llm.stream(history_langchain_format)
    full_content = ""
    for chunk in stream:
        if chunk.content:
            full_content += chunk.content
            yield full_content

# ✅ Query Expansion (스트림)
def expand_query(user_query):
    expansion_prompt = f"""아래 사용자의 쿼리를 더 명확하고 구조화된 프롬프트로 확장해줘. 최대한 자세하게 작성하고 바로 복사 가능한 형태로 작성해줘. 설명을 제외하고 유저가 바로 복사해서 붙여넣기 할 프롬프트는 [변경된 프롬프트] 아래에 작성해줘.
사용자 쿼리:
{user_query}
"""
    stream = llm.stream([HumanMessage(content=expansion_prompt)])
    full_content = ""
    for chunk in stream:
        if chunk.content:
            full_content += chunk.content
            yield full_content

system_prompt = """당신은 인공지능 모델로서 어린 초등학생의 질문을 받게 됩니다. 초등학생의 질문에 대해 최대한 자세하고 풍부한 설명이 필요합니다. 또한 친절하고 친근한 말투를 유지할 수 있도록 하세요. 가능한 경우 이모지를 추가하여 친근함을 더하도록 합니다."""

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # ✅ 시스템 프롬프트: 상단 고정
    extra_input = gr.Textbox(
        value=system_prompt,
        label="시스템 프롬프트"
    )
    system_prompt_state = gr.State(system_prompt)

    # ✅ 검색 토글 → 유저 입력창 바로 아래
    search_toggle = gr.Checkbox(label="인터넷 검색 사용 (SerpAPI)", value=False)

    chat = gr.ChatInterface(
        fn=response,
        additional_inputs=[system_prompt_state, search_toggle],
        title="나만의 챗봇",
        description="나만의 챗봇으로 Customizing하여 대화하기!",
        submit_btn="보내기 📨",
        stop_btn="멈추기 ⏹️",
    )

    gr.Examples(
        examples=[
            ["세계에서 가장 높은 산은?"],
            ["최신 AI 뉴스 알려줘"],
            ["오늘 서울 날씨 어때?"]
        ],
        inputs=chat.textbox,
    )

    # ✅ Query Expansion
    with gr.Accordion(label="💡 Query Expansion (LLM 기반)", open=True):
        user_input = gr.Textbox(label="원본 메시지")
        expanded_output = gr.Textbox(label="확장된 메시지", interactive=True)
        expand_btn = gr.Button("Query Expansion 실행")
        expand_btn.click(
            fn=expand_query,
            inputs=user_input,
            outputs=expanded_output
        )

demo.launch()
